# [1633. Percentage of Users Attended a Contest](https://leetcode.com/problems/percentage-of-users-attended-a-contest/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Users

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| user_id     | int     |
| user_name   | varchar |
+-------------+---------+</pre>
user_id is the primary key (column with unique values) for this table.
Each row of this table contains the name and the id of a user.


Table: Register

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| contest_id  | int     |
| user_id     | int     |
+-------------+---------+</pre>
(contest_id, user_id) is the primary key (combination of columns with unique values) for this table.
Each row of this table contains the id of a user and the contest they registered into.


Write a solution to find the percentage of the users registered in each contest rounded to two decimals.

Return the result table ordered by percentage in descending order. In case of a tie, order it by contest_id in ascending order.

The result format is in the following example.



Example 1:

Input:
Users table:
<pre>+---------+-----------+
| user_id | user_name |
+---------+-----------+
| 6       | Alice     |
| 2       | Bob       |
| 7       | Alex      |
+---------+-----------+</pre>
Register table:
<pre>+------------+---------+
| contest_id | user_id |
+------------+---------+
| 215        | 6       |
| 209        | 2       |
| 208        | 2       |
| 210        | 6       |
| 208        | 6       |
| 209        | 7       |
| 209        | 6       |
| 215        | 7       |
| 208        | 7       |
| 210        | 2       |
| 207        | 2       |
| 210        | 7       |
+------------+---------+</pre>
Output:
<pre>+------------+------------+
| contest_id | percentage |
+------------+------------+
| 208        | 100.0      |
| 209        | 100.0      |
| 210        | 100.0      |
| 215        | 66.67      |
| 207        | 33.33      |
+------------+------------+</pre>
Explanation:
All the users registered in contests 208, 209, and 210. The percentage is 100% and we sort them in the answer table by contest_id in ascending order.
Alice and Alex registered in contest 215 and the percentage is ((2/3) * 100) = 66.67%
Bob registered in contest 207 and the percentage is ((1/3) * 100) = 33.33%

In [2]:
#pandas schema

import pandas as pd

data = [[6, 'Alice'], [2, 'Bob'], [7, 'Alex']]
users = pd.DataFrame(data, columns=['user_id', 'user_name']).astype({'user_id': 'Int64', 'user_name': 'object'})
data = [[215, 6], [209, 2], [208, 2], [210, 6], [208, 6], [209, 7], [209, 6], [215, 7], [208, 7], [210, 2], [207, 2],
        [210, 7]]
register = pd.DataFrame(data, columns=['contest_id', 'user_id']).astype({'contest_id': 'Int64', 'user_id': 'Int64'})

# to spark dataframe

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

users_df = spark.createDataFrame(users)
register_df = spark.createDataFrame(register)

In [3]:
users
register

,contest_id,user_id
0,215,6
1,209,2
2,208,2
3,210,6
4,208,6
5,209,7
6,209,6
7,215,7
8,208,7
9,210,2


In [11]:
# solving using spark dataframe
t=users_df.count()

register_df\
        .join(users_df,['user_id'],'inner')\
        .groupBy('contest_id').agg(round(count('contest_id')*100/t ,2).alias('percentage'))\
        .orderBy(col('percentage').desc(),col('contest_id').asc())\
        .show()

+----------+----------+
|contest_id|percentage|
+----------+----------+
|       208|     100.0|
|       209|     100.0|
|       210|     100.0|
|       215|     66.67|
|       207|     33.33|
+----------+----------+



In [ ]:
# solving in spark sql

users_df.createOrReplaceTempView('users')
register_df.createOrReplaceTempView('register')

spark.sql('''
select
        contest_id,
        round(100*count(register.user_id)/(select count(user_id) from users),2) as percentage
from register
inner join users
on register.user_id = users.user_id
group by contest_id
order by percentage desc, contest_id asc;
''').show()

+----------+----------+
|contest_id|percentage|
+----------+----------+
|       208|     100.0|
|       209|     100.0|
|       210|     100.0|
|       215|     66.67|
|       207|     33.33|
+----------+----------+


In [ ]:
spark.stop()